In [1]:
import os
import tensorflow as tf
import numpy as np

# Set the seed for random operations. 
# This let our experiments to be reproducible. 
SEED = 2112
tf.random.set_seed(SEED)  

from google.colab import drive
drive.mount("/content/drive")

# Get current working directory
cwd = os.getcwd()

Mounted at /content/drive


In [ ]:
!unzip '/content/drive/My Drive/challenge/artificial-neural-networks-and-deep-learning-2020.zip'

In [18]:
dataset_dir = os.path.join(cwd, 'MaskDataset')
dataset_dir

'/content/MaskDataset'

In [4]:
import json
import os
import shutil


with open("./MaskDataset/train_gt.json", "r") as f:
  data = json.load(f)

os.mkdir("./MaskDataset/new_train") 
os.mkdir("./MaskDataset/new_train/0")
os.mkdir("./MaskDataset/new_train/1")
os.mkdir("./MaskDataset/new_train/2")
os.mkdir("./MaskDataset/new_val")
os.mkdir("./MaskDataset/new_val/0")
os.mkdir("./MaskDataset/new_val/1")
os.mkdir("./MaskDataset/new_val/2")


counter_0 = 0
counter_1 = 0
counter_2 = 0

path_dataset = "./MaskDataset/training/" #/content/MaskDataset/training
path_new_train = "./MaskDataset/new_train/" #/content/MaskDataset/new_train/
path_new_val = "./MaskDataset/new_val/" #/content/MaskDataset/new_val/

for attribute, value in data.items():
        if value == 0:
            counter_0 += 1 
            if (counter_0 < 1400):
                shutil.copy(path_dataset + attribute,path_new_train + "0/" + attribute)
            else:
                shutil.copy(path_dataset + attribute,path_new_val + "0/" + attribute)
        elif value == 1:
            counter_1 += 1 
            if (counter_1 < 1400):
                shutil.copy(path_dataset + attribute,path_new_train + "1/" + attribute)
            else:
                shutil.copy(path_dataset + attribute,path_new_val + "1/" + attribute)
        else:
            counter_2 += 1 
            if (counter_2 < 1400):
                shutil.copy(path_dataset + attribute,path_new_train + "2/" + attribute)
            else:
                shutil.copy(path_dataset + attribute,path_new_val + "2/" + attribute)

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
img_h = 512
img_w = 512
num_classes = 3

# Batch size
bs = 8
train_data_gen = ImageDataGenerator(rescale=1/255., rotation_range=10,
                                    width_shift_range=10,
                                    height_shift_range=10,
                                    zoom_range=0.3,
                                    horizontal_flip=True,
                                    vertical_flip=True,
                                    fill_mode='constant',
                                    cval=0,)
valid_data_gen = ImageDataGenerator(rescale=1/255.)

train_gen=train_data_gen.flow_from_directory("./MaskDataset/new_train",class_mode='categorical',shuffle=True, batch_size=bs, target_size=(img_h,img_w)) 
valid_gen = valid_data_gen.flow_from_directory("./MaskDataset/new_val", class_mode='categorical', shuffle=True, batch_size=bs, target_size=(img_h,img_w))


Found 4197 images belonging to 3 classes.
Found 1417 images belonging to 3 classes.


In [8]:
#params
img_h = 512
img_w = 512
num_classes = 3

# Batch size
bs = 8


In [9]:
# Create Dataset objects

# Training
train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))
train_dataset = train_dataset.repeat()

# Validation
valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen, 
                                               output_types=(tf.float32, tf.float32),                  
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))
valid_dataset = valid_dataset.repeat()

In [10]:


mb = tf.keras.applications.MobileNet(weights='imagenet', include_top=False, input_shape=(img_h, img_w, 3)) #include_top false perchè tolgo il classifier 
#weights uso quelli di imagenet che sono i più usati

17227776/17225924 [==============================] - 0s 0us/step


In [11]:
# Create Model
# ------------

finetuning = True

#finetuning true significa che da un certo livello in poi ricomincio ad adattare i pesi, partendo da quelli dati
#di solito si fa su modelli molto diversi da quello usato in origine

if finetuning:
    freeze_until = 15 # layer from which we want to fine-tune
    
    for layer in mb.layers[:freeze_until]:
        layer.trainable = False
else:
    mb.trainable = False
    
model = tf.keras.Sequential()
model.add(mb)
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=1024, activation='relu'))
model.add(tf.keras.layers.Dense(units=512, activation='relu'))
model.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))

In [12]:
# Optimization params
# -------------------

# Loss
loss = tf.keras.losses.CategoricalCrossentropy()

# learning rate
lr = 1e-4
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
# -------------------

# Validation metrics
# ------------------

metrics = ['accuracy']
# ------------------

# Compile Model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [14]:
callbacks = []

# Early Stopping
early_stop = True
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    callbacks.append(es_callback)

model.fit_generator(train_gen, epochs=100, validation_data=valid_gen, callbacks=callbacks)
    


Epoch 1/100
525/525 [==============================] - 262s 500ms/step - loss: 0.4411 - accuracy: 0.8087 - val_loss: 0.3166 - val_accuracy: 0.8793
Epoch 2/100
525/525 [==============================] - 263s 501ms/step - loss: 0.4003 - accuracy: 0.8287 - val_loss: 0.2832 - val_accuracy: 0.8878
Epoch 3/100
525/525 [==============================] - 262s 500ms/step - loss: 0.3012 - accuracy: 0.8751 - val_loss: 0.2521 - val_accuracy: 0.9054
Epoch 4/100
525/525 [==============================] - 263s 502ms/step - loss: 0.2640 - accuracy: 0.8921 - val_loss: 0.2114 - val_accuracy: 0.9231
Epoch 5/100
525/525 [==============================] - 261s 497ms/step - loss: 0.2407 - accuracy: 0.9064 - val_loss: 0.2254 - val_accuracy: 0.9061
Epoch 6/100
525/525 [==============================] - 260s 495ms/step - loss: 0.2331 - accuracy: 0.9078 - val_loss: 0.3009 - val_accuracy: 0.8864
Epoch 7/100
525/525 [==============================] - 258s 491ms/step - loss: 0.2037 - accuracy: 0.9190 - val_loss: 0

In [15]:
import os
from datetime import datetime

def create_csv(results, results_dir='./'):

    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(os.path.join(results_dir, csv_fname), 'w') as f:

        f.write('Id,Category\n')

        for key, value in results.items():
            f.write(key + ',' + str(value) + '\n')

In [17]:
results = {}
for i in os.listdir("/content/MaskDataset/test"):
  img = tf.keras.preprocessing.image.load_img("/content/MaskDataset/test/"+i, target_size=(512,512), color_mode='rgb')
  img_array = tf.keras.preprocessing.image.img_to_array(img)
  img_array = img_array/255.0
  img_array = tf.expand_dims(img_array,0)
  predictions = model.predict(img_array)
  score = tf.nn.softmax(predictions[0])
  predicted_class_int = np.argmax(predictions[0])
  results[i] = predicted_class_int
create_csv(results)